# Parkinson's disease progression notebook

### Motivatie en originaliteit van de gebruikte methoden


Het voorspellen van het verloop van de ziekte van Parkinson (PD) op basis van eiwitten uit onze gegeven is een complexe taak die een doordachte aanpak vereist. Hier zijn de stappen en overwegingen die wij hebben doorlopen voor het kiezen van het juiste model.

* Data Preprocessing:
    -   Wij zijn begonnen met het opgschonenen en voorbereiden van onze data, Hierbij hebben wij processen gebruikte als groeperen, piviton en mergen.


In [ ]:
def prepare_dataset(train_proteins: pd.DataFrame, train_peptides: pd.DataFrame):

    # Grouping 
    df_protein_grouped = train_proteins.groupby(["visit_id","UniProt"])["NPX"].mean().reset_index()
    df_peptide_grouped = train_peptides.groupby(["visit_id","Peptide"])["PeptideAbundance"].mean().reset_index()

    # Pivoting
    df_protein = df_protein_grouped.pivot(index="visit_id", columns="UniProt", values="NPX").rename_axis(columns=None).reset_index()
    df_peptide = df_peptide_grouped.pivot(index="visit_id", columns="Peptide", values="PeptideAbundance").rename_axis(columns=None).reset_index()
    
    # Merging
    pro_pep_df = df_protein.merge(df_peptide, on=["visit_id"], how="left")
    
    return pro_pep_df

### Documentation

### Kaggle score